In [10]:
import sys
sys.path.append('../../30_data_tools/')
import sqlite3
import pandas as pd
from PIL import Image, ImageFont, ImageDraw
from tqdm.auto import tqdm
from pathlib import Path

In [2]:
from helper import load_dotenv, get_pdf_page_processing_status
from file_interaction import blob_exists, open_img, get_generic_image_filepath, get_data_files

In [3]:
from file_interaction import get_blobs

In [4]:
import re

In [11]:
Image.MAX_IMAGE_PIXELS = None

In [5]:
def generate_marker_image( pdf_row ):
    filepath = get_generic_image_filepath(
        pdf_row.pdf_filename,
        pdf_row.job,
        pdf_row.method,
        pdf_row.idx
    )

    if filepath:
        img = open_img( filepath ).convert('RGB')

        # Masken laden
        c = con.cursor()
        sql_string = f'''
            SELECT bbox,ssim FROM mask
            WHERE 
                pdf_filename='{ pdf_row.pdf_filename }' AND
                job='{ pdf_row.job }' AND
                "type"='{ pdf_row["type"] }' AND
                variant_name='{ pdf_row.variant_name }' AND
                method='{ pdf_row.method }' AND
                idx='{ pdf_row.idx }'
        '''
        c.execute(sql_string)
        masks = c.fetchall()
        c.close

        for bbox_string,ssim_value in masks:
            bbox = [int(val) for val in bbox_string.split(';')]
            color = (255,0,0,255) if ssim_value < 0.95 else (0,0,255,255)
            font = ImageFont.load_default(size=50)      
            d = ImageDraw.Draw(img, 'RGBA')
            d.rectangle(
                [bbox[0],bbox[1],bbox[0]+bbox[2],bbox[1]+bbox[3]],
                outline=color,
                width=10
            )
            d.text((bbox[0],bbox[1]), str(ssim_value), fill=color, font=font)

        out_path = f'./10_marker_images/{ pdf_row.job }.{ pdf_row.pdf_filename }.{ pdf_row.method }.{ pdf_row.idx }.marker.jpg'
        
        img.save( out_path, progressive=True, dpi=( dotenv['LOFI_DPI'], dotenv['LOFI_DPI'] ) )

In [6]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [7]:
pages = pd.read_sql(
    '''
        SELECT * FROM generic_image
        WHERE job='24-03-05-01_randomTrainPages'
    ''',
    con,
    parse_dates=['timestamp']
)

pages.loc[
    :,
    'already_processed'
] = pages.apply(lambda pdf_row: Path(f'./10_marker_images/{ pdf_row.job }.{ pdf_row.pdf_filename }.{ pdf_row.method }.{ pdf_row.idx }.marker.jpg').exists(), axis=1 ) == False

In [8]:
f'{ pages.loc[pages.already_processed].shape[0] }/{ pages.shape[0] }'

'132/324'

In [9]:
0 / 0

ZeroDivisionError: division by zero

In [13]:
relevant_idx = pages.loc[pages.already_processed].index

for idx in tqdm(relevant_idx):
    row = pages.loc[idx]

    generate_marker_image( row )

  0%|          | 0/132 [00:00<?, ?it/s]